In [ ]:
import pandas as pd
#settings :
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno 
import datatable as dt  # pip install datatble
import datetime
from pandas.tseries.holiday import Holiday,AbstractHolidayCalendar
#to create russian holidays list
!pip install holidays
import holidays
from datetime import date
#!pip install googletrans




<p style="background-color:green ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
    &nbsp; &nbsp; &nbsp; &nbsp;<b>GOAL :</b>
    Collect historical data related to previous sales and use it to predict expected sales.</p>
    
 


<h4 style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
    &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Data fields </h4>

<ul style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
 <li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;ID - an Id that represents a (Shop, Item) tuple within the test set</li>
<li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;shop_id - unique identifier of a shop</li>
<li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;item_id - unique identifier of a product</li>
<li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;item_category_id - unique identifier of item category</li>
<li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;item_cnt_day - number of products sold. You are predicting a monthly amount of this measure</li>
<li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;item_price - current price of an item</li>
<li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;date - date in format dd/mm/yyyy</li>
<li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33</li>
<li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;item_name - name of item</li>
<li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;shop_name - name of shop</li>
<li>&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;item_category_name - name of item category</li>
    </ul>

In [ ]:
data = pd.read_csv('../input/aipro2predictsales/all_data_competition_en.csv')

In [ ]:
data.head(5)

<ul style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
&nbsp; &nbsp; &nbsp;Changing the date format 

    pandas datetime format is YYYY-MM-DD
</ul>

In [ ]:
data['date'] = pd.to_datetime(data['date'], infer_datetime_format=True)  

In [ ]:
data.head()

<h4 style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
    &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Adding more more specific column for date time such as :</h4>
  <ul style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
    <li> week_day : Monday=0 ,Sunday =6</li>

<li> day : day(1-31)</li>
 
<li> month : month (1-12)</li>

<li> year : year </li>
 </ul>


In [ ]:
data['week_day']= data['date'].dt.dayofweek
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year


<h4 style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
    &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;HOW MANHY YEARS CONTAINS IN THIS DATASET ?</h4>

In [ ]:
data['year'].value_counts()



<h4 style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
    &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;Finding the earliest and latest date of the dataset</h4>

In [ ]:
print('oldest date is :',data.date.min())
print('latest date is :',data.date.max())


<h4 style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
    &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  Generating Russion Holidays </h4>

<p>     I will be using holidays libaray from this : https://pypi.org/project/holidays/

        by installing with :

<span style="background-color:lightgray ;color :black;,font-size:12em; border-radius: 20px; display: inline-block; text-align: center-right; padding: 20px ">
  !pip install holidays</span></p>




<h4 style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
    &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;Step 1 : Creating a holidays lists</h4>

In [ ]:
russion_holiday_lists=[]
russion_holiday_name=[]

for day,name in sorted(holidays.Russia(years=[2013,2014,2015]).items()):
    #print(day,name)
    russion_holiday_lists.append(day)
    russion_holiday_name.append(name)
    

#how many holidays in total 3 years
print("total holidays in 3 yrs : ",len(russion_holiday_lists))




<h4 style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
    &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;Step 2 : Creating a seperate holiday dataframe</h4>

In [ ]:

rus_hol ={
    'rus_hol':russion_holiday_lists,
    'hol_name_ru':russion_holiday_name

}
holidays_dataset=pd.DataFrame(rus_hol)
holidays_dataset['rus_hol'] = pd.to_datetime(holidays_dataset['rus_hol'], infer_datetime_format=True) 

"""#since the holidays name are in russia I will be translating it to english
from googletrans import Translator


x=holidays_dataset['hol_name_ru']
translator = Translator()#defining translator
holidays_dataset['hol_name_en'] = holidays_dataset['hol_name_ru'].apply(lambda x: translator.translate(x, dest='en').text)
#df['translated_value'] = df['value'].apply(lambda x: translator.translate(x, dest='en').text)

#changing the format to datetime 
#df = pd.to_datetime(df['rus_hol'], infer_datetime_format=True)  """

holidays_dataset.sort_index(inplace=True)
holidays_dataset



<h5 style="background-color:darkgray ;color :white;,font-size:12em; border-radius: 20px; text-align: center-right; padding: 20px">
    &nbsp; &nbsp; &nbsp; &nbsp;Adding a new column  to our original dataset called <b> 'is_holiday_dich' </b>&nbsp; &nbsp;1 = yes ,&nbsp; &nbsp; 0 = no</h5>
    
 

    


In [ ]:

data['is_holiday_dich'] = data['date'].isin(holidays_dataset['rus_hol'])
#chaning from true or false to a coloumns of integers 1 and 0 

data['is_holiday_dich']=data['is_holiday_dich'].astype(int)
data.head()

In [ ]:
data['is_holiday_dich'].value_counts()